<a href="https://colab.research.google.com/github/lilianabs/learning-pytorch/blob/main/Simple_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the code from this tutorial: https://coderzcolumn.com/tutorials/artificial-intelligence/guide-to-create-simple-neural-networks-using-pytorch

In [1]:
import torch

print("PyTorch Version : {}".format(torch.__version__))

PyTorch Version : 1.13.1+cu116


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device : {}".format(device))

Device : cuda


In [7]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [8]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

In [9]:
#X, y = datasets.load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=123)

X_train, X_test, y_train, y_test = torch.tensor(X_train, dtype=torch.float32),\
                                   torch.tensor(X_test, dtype=torch.float32),\
                                   torch.tensor(y_train, dtype=torch.float32),\
                                   torch.tensor(y_test, dtype=torch.float32)

samples, features = X_train.shape


In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([404, 13]),
 torch.Size([102, 13]),
 torch.Size([404]),
 torch.Size([102]))

In [14]:
# Normalize the data
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [15]:
def initialize_weights(layer_sizes):
    weights = []
    for i, units in enumerate(layer_sizes):
        if i==0:
            w = torch.rand(units,features, dtype=torch.float32, requires_grad=True) ## First Layer
        else:
            w = torch.rand(units,layer_sizes[i-1], dtype=torch.float32, requires_grad=True) ## All other layers

        b = torch.rand(units, dtype=torch.float32, requires_grad=True) ## Bias

        weights.append([w,b])

    return weights

In [16]:
torch.manual_seed(123)

weights = initialize_weights([15,10,1])

for i, (w,b) in enumerate(weights):
    print("Layer : {}, Weights : {}, Biases : {}".format(i+1, w.shape, b.shape))

Layer : 1, Weights : torch.Size([15, 13]), Biases : torch.Size([15])
Layer : 2, Weights : torch.Size([10, 15]), Biases : torch.Size([10])
Layer : 3, Weights : torch.Size([1, 10]), Biases : torch.Size([1])


In [17]:
def Relu(tensor):
    return torch.maximum(tensor, torch.zeros_like(tensor)) # max(0,x)

In [18]:
tensor = torch.tensor([-1,0,1,-2,4,-6,5])

Relu(tensor)

tensor([0, 0, 1, 0, 4, 0, 5])

In [19]:
def LinearLayer(weights, input_data, activation=lambda x: x):
    w, b = weights
    out = torch.matmul(input_data, w.T) + b ## Multiply input by weights and add bias to it.
    return activation(out) ## Apply activation at last

In [20]:
rand_data = torch.rand(5, features)

out = LinearLayer(weights[0], rand_data, Relu)

print("Data Shape : {}".format(rand_data.shape))
print("Output Shape : {}".format(out.shape))

Data Shape : torch.Size([5, 13])
Output Shape : torch.Size([5, 15])


In [ ]:
def forward(weights, input_data):
    layer_out = input_data

    for i in range(len(weights[:-1])):
        layer_out = LinearLayer(weights[i], layer_out, Relu) ## Hidden Layer

    preds = LinearLayer(weights[-1], layer_out) ## Final Layer

    return preds.ravel()